In [ ]:
import time
import string
import nltk
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pyhdb

start_time = time.time()

connection = pyhdb.connect(host="IP", port=30015, user="USERNAME", password="PASS")
query = "SQL QUERY TO PULL DATA"
dtProductDetails = pd.read_sql_query(query, connection)
dtProductDetails.head()

dtProductDetails.dropna(inplace=True)
df = dtProductDetails[["PRODUCT_CODE", "JIOMART_PRODUCT_ID", "DESC_DTL", "CATEGORY"]]
df

# to remove data in HTML tags
#import re
#CLEANR = re.compile('<.*?>|<img [^>]*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

from bs4 import BeautifulSoup
def cleanhtml(raw_html):
    cleantext = BeautifulSoup(raw_html, "lxml").text
    return cleantext

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(cleanhtml)
df

stopwords = nltk.corpus.stopwords.words('english')
def clean(doc): # doc is a string of text
    doc = doc.replace("</br>", " ")
    doc = "".join([char if char not in string.punctuation else " " for char in doc])
    #doc = "".join([char for char in doc if char not in string.punctuation])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    return doc.lower()

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(clean)
df

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def fnLemmatize(doc): # doc is a string of text
    word_list = nltk.word_tokenize(doc)
    doc = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return doc

df["DESC_DTL"] = df["DESC_DTL"].astype(str).apply(fnLemmatize)

df = df[df.DESC_DTL != '']

from keybert import KeyBERT
kb_model = KeyBERT()

# Custom Function - Extract Keyword
def extractKeyword (txt):
    keyword = kb_model.extract_keywords(txt, keyphrase_ngram_range=(1,2), stop_words='english', highlight=False, top_n=30, diversity=0.7)
    if len(keyword) > 0:
        keywords = list(zip(*keyword))[0]
        strKeywords = ','.join(str(item) for item in keywords)
        
    if len(keyword) == 0:
        return None
    else:
        return strKeywords

df["keywords"] = df["DESC_DTL"].astype(str).apply(extractKeyword)

end_time = time.time()

elapsed_time = end_time - start_time
print('Execution time:', elapsed_time, 'seconds')